In [29]:
import jax 
import jax.random as random

import brax
from brax import envs
from experior.envs import BraxGymnaxWrapper, ClipAction
from experior.utils import moving_average
from experior.rl_agents.utils import ContinuousActorCritic
from experior.rl_agents import make_continuous_ppo_train
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from experior.envs import ContinuousMountainCar

## Environment

In [41]:
# Humanoid environment
env = ClipAction(BraxGymnaxWrapper('humanoid'), low=-1.0, high=1.0)
#env = ContinuousMountainCar()

## PPO

In [51]:
ppo_config = {
  "learning_rate": 2.5e-4,
  "steps": 1e5,
  "train_frequency": 128,
  "num_envs": 3,
  "num_actors": 7,
  "max_grad_norm": 0.5,
  "epochs_per_iteration": 4,
  "num_minibatches": 4,
  "ent_coef": 0.01,
}

In [54]:
actor_critic_network = ContinuousActorCritic(#
    action_dim=17,
    activation='tanh'
)

ppo_train = make_continuous_ppo_train(
    env=env,
    actor_critic_network=actor_critic_network,
    **ppo_config
)

In [55]:
jax.config.update("jax_disable_jit", False)
ppo_train = jax.jit(ppo_train)
state, ppo_output = ppo_train(jax.random.PRNGKey(42))

UnexpectedTracerError: Encountered an unexpected tracer. A function transformed by JAX had a side effect, allowing for a reference to an intermediate value with type float32[1,7,17] wrapped in a BatchTracer to escape the scope of the transformation.
JAX transformations require that functions explicitly return their outputs, and disallow saving intermediate values to global state.
To catch the leak earlier, try setting the environment variable JAX_CHECK_TRACER_LEAKS or using the `jax.checking_leaks` context manager.Detail: Different traces at same level: Traced<ShapedArray(float32[1,7,17])>with<BatchTrace(level=4/0)> with
  val = Traced<ShapedArray(float32[1,1,7,17])>with<DynamicJaxprTrace(level=3/0)>
  batch_dim = 0, BatchTrace(level=4/0)
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.UnexpectedTracerError

In [17]:
env.observation_size

(244,)